In [1]:
#线性回归的简洁实现
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l


In [2]:
#生成数据集
true_w = torch.tensor([2, -3.4])
true_b = 4.2
#synthetic_data Generate y = Xw + b + noise.
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [3]:
#读取数据集
#is_train表⽰是否希望数据迭代器对象在每个迭代周期内打乱数据,shuffle 打乱
def load_array(data_arrays, batch_size, is_train=True): #@save
    """构造⼀个PyTorch数据迭代器"""  #dataset 用于打包和预处理
    dataset = data.TensorDataset(*data_arrays) #数据集包装张量，每个样本将通过沿第一索引张量检索
    return data.DataLoader(dataset, batch_size, shuffle=is_train) 
#TensorDataset 对 tensor 以类似于一对一分配的方式进行组装、打包，该类通过每一个 tensor 的第一个维度进行索引。
#DataLoader可以将组装好的张量以希望的方式输出
#DataLoader 将整个资料集（dataset）按照batch进行迭代分装或者shuffle（可以得到一个iterator以利于for循环读取）

In [4]:
batch_size = 10 #批大小
data_iter = load_array((features, labels), batch_size) #迭代器，加载数据集

In [5]:
#验证data_iter是否正常工作，打印并读取第一个小批量样本并使用next从迭代器中获取第一项
#next() 返回迭代器的下一个项目，该函数要和生成迭代器的iter()函数一同使用
#iter()函数实际上就是调⽤了可迭代对象的 iter ⽅法  iteration
next(iter(data_iter))

[tensor([[-2.4956, -0.1143],
         [ 1.0424,  0.4546],
         [ 0.5344, -0.1414],
         [-0.1405,  1.6028],
         [ 0.1231,  1.0679],
         [ 0.3005, -0.3298],
         [-0.1562, -0.4190],
         [ 0.4543, -1.2037],
         [ 0.0903,  0.7921],
         [-0.0310, -0.6457]]),
 tensor([[-0.4047],
         [ 4.7350],
         [ 5.7307],
         [-1.5050],
         [ 0.8141],
         [ 5.9252],
         [ 5.3007],
         [ 9.2138],
         [ 1.6890],
         [ 6.3326]])]

In [7]:
#定义模型
#Sequential类 能将多个层串联在一起，上一层的输入作为下层输出
# nn是神经⽹络的缩写
'''
torch.nn.Linear 类用于定义模型的线性层，即完成前面提到的不同的层之间的线性变换
torch.nn.Linear(in_features, # 输入的神经元个数
           out_features, # 输出神经元个数
           bias=True # 是否包含偏置
           )
'''
'''
torch.nn.Sequential 类是 torch.nn 中的一种序列容器，
通过在容器中嵌套各种实现神经网络中具体功能相关的类，
来完成对神经网络模型的搭建，
最主要的是，参数会按照我们定义好的序列自动传递下去
'''
from torch import nn
net = nn.Sequential(nn.Linear(2, 1)) #生成一个线性层


In [9]:
#初始化模型参数
net[0].weight.data.normal_(0, 0.01) #net[0] 选择⽹络中的第⼀个图层，使用正态分布填充 weight.data
net[0].bias.data.fill_(0) #使⽤替换⽅法 fill_来重写参数值

tensor([0.])

In [10]:
#定义损失函数
loss = nn.MSELoss() #mean squared error 均方误差

In [11]:
#定义优化函数 SGD 随机梯度下降
trainer = torch.optim.SGD(net.parameters(), lr=0.03) # net.parameters() 需要优化的参数，lr 学习率

In [13]:
#训练
'''
回顾⼀下：在每个迭代周期⾥，我们将完整遍历⼀次数据集（train_data），不停地从中获取⼀个⼩批量的
输⼊和相应的标签。对于每⼀个⼩批量，我们会进⾏以下步骤:
• 通过调⽤net(X)⽣成预测并计算损失l（前向传播）。
• 通过进⾏反向传播来计算梯度。
• 通过调⽤优化器来更新模型参数。
'''
num_epochs = 3 #迭代周期数
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y) #损失函数
        trainer.zero_grad() #优化函数 遍历模型所有参数，截断反向传播的梯度流并将每个参数的梯度值设为0，即清空上次的梯度记录
        l.backward()        #反向传播，产生梯度，通过autograd包实现，根据tensor进行过的数学运算来自动计算其对应的梯度
        trainer.step()      #执行一次优化步骤，通过梯度下降执行一步参数更新，执行optimizer.step()函数前应先执行loss.backward()函数来计算梯度
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000189
epoch 2, loss 0.000101
epoch 3, loss 0.000101


In [14]:
w = net[0].weight.data #从net访问所需的层，然后读取该层的权重
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data #读取偏置
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0004, 0.0008])
b的估计误差： tensor([0.0006])


In [ ]:
'''
小结：
• 我们可以使⽤PyTorch的⾼级API更简洁地实现模型。
• 在PyTorch中，data模块提供了数据处理⼯具，nn模块定义了⼤量的神经⽹络层和常⻅损失函数。
• 我们可以通过_结尾的⽅法将参数替换，从⽽初始化参数。
'''